In [ ]:
import subprocess
from time import sleep

import numpy as np
import pynvml
from tqdm import tqdm
import time
import os

pynvml.nvmlInit()
last_task_t = [0] * 10


def get_free_MB(i):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    return pynvml.nvmlDeviceGetMemoryInfo(handle).free / 1024 / 1024


def get_ps_num(i):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    gpu_ps = pynvml.nvmlDeviceGetComputeRunningProcesses(handle)
    return len(gpu_ps)


def get_avail_gpu(size=10000):
    num = pynvml.nvmlDeviceGetCount()
    free = []
    for _ in range(60):
        sleep(1)
        free.append([get_free_MB(i) for i in range(num)])
    free = np.array(free).T.tolist()
    avail_gpus = []
    for i in range(num):
        if i == 3:
            continue
        if min(free[i]) > size and time.time() - last_task_t[i] > 600:
            avail_gpus.append(i)
    return avail_gpus

In [ ]:
tasks = []
# for tol in [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]:
#     tasks.append((f"wfgan_tol{tol}", "wfgan"))
for gan in [1, 3, 5, 7, 9]:
    for tol in [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]:
        tasks.append((f"switch_gan{gan:02d}_tol{tol}_front{10-gan:02d}", "switch_gan_front"))
# for p in ["equal", "random"]:
#     for tamaraw in range(2):
#         for regulartor in range(2):
#             for front in range(2):
#                 for wfgan in range(2):
#                     if tamaraw + regulartor + front + wfgan > 0:
#                         name = f"switch_{p}"
#                         if front:
#                             name += "_front"
#                         if wfgan:
#                             name += "_wfgan"
#                         if tamaraw:
#                             name += "_tamaraw"
#                         if regulartor:
#                             name += "_regulartor"
#                         tasks.append((name, f"switch_{p}"))


# tasks.extend([(name, "base") for name in ["undefend", "front", "tamaraw", "wfgan", "regulartor"]])

In [ ]:
avail_gpus = []
for task, note in tqdm(tasks):
    log_dir = f"run/20240308{note}/RF/train_undefend_d{task}"
    if os.path.exists(log_dir):
        print(f"skip {task}")
        continue
    while not avail_gpus:
        avail_gpus = get_avail_gpu()

    gpu = avail_gpus.pop()
    subprocess.Popen(
        f"python train.py --train {task} --note {note} -g {gpu}",
        shell=True,
        stdout=subprocess.DEVNULL,
    )
    last_task_t[gpu] = time.time()